MIS 285N Cognitive Computing<br>
Final Project<br>
Jerry Che - Jose Guerrero - Riley Moynihan - Noah Placke - Sarah Teng - Palmer Wenzel

# Data Preprocessing (Kaggle Source)

#### Read data from CSV.

In [1]:
import pandas as pd
# pd.options.display.max_columns = 500


df_raw = pd.read_csv('raw/RAW_recipes.csv').sample(frac=0.1, random_state=0)

df_raw.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
99795,grilled salmon with rosemary south beach phase i,479456,22,1682431,2012-05-15,"['30-minutes-or-less', 'time-to-make', 'course...","[167.0, 11.0, 0.0, 9.0, 46.0, 6.0, 0.0]",10,"['cut the fish into 4 equal-size portions', 'c...","a delicious, phase-1-friendly dinner for fello...","['salmon', 'olive oil', 'fresh lemon juice', '...",9
155700,peanut butter and jelly monte cristo sandwiches,219246,20,9717,2007-03-27,"['30-minutes-or-less', 'time-to-make', 'course...","[551.4, 54.0, 60.0, 25.0, 25.0, 85.0, 16.0]",8,['spread 1 tablespoons peanut butter on each o...,did you know that american gi's are credited w...,"['creamy peanut butter', 'strawberry jelly', '...",7
152789,parmesan baked asparagus,298592,24,293757,2008-04-16,"['30-minutes-or-less', 'time-to-make', 'course...","[72.1, 6.0, 6.0, 4.0, 10.0, 7.0, 1.0]",6,"['preheat ovento 425 f', 'place the asparagus ...",a tasty twist that is easy and low fat....just...,"['asparagus', 'extra virgin olive oil', 'seaso...",5
95519,grandmother s old time tea cakes,191469,35,351995,2006-10-23,"['60-minutes-or-less', 'time-to-make', 'course...","[297.5, 17.0, 136.0, 5.0, 6.0, 14.0, 15.0]",11,['mix shortening and sugar together with pastr...,my grandmother would always have a plastic tub...,"['flour', 'sugar', 'eggs', 'buttermilk', 'shor...",8
121315,lemon chicken with olives,245476,50,108488,2007-08-08,"['lactose', '60-minutes-or-less', 'time-to-mak...","[229.8, 24.0, 18.0, 14.0, 26.0, 19.0, 3.0]",11,"['fry off chicken until browned , about 5 minu...",from super food ideas magazine(july 2007). be ...,"['boneless chicken thighs', 'olive oil', 'onio...",10


#### Drop unnecessary columns.

In [2]:
df_dropped = df_raw.drop(['id', 'minutes', 'contributor_id', 'submitted', 'nutrition', 'n_steps', 'description', 'n_ingredients', 'tags'], axis=1)

df_dropped.head()

,name,steps,ingredients
99795,grilled salmon with rosemary south beach phase i,"['cut the fish into 4 equal-size portions', 'c...","['salmon', 'olive oil', 'fresh lemon juice', '..."
155700,peanut butter and jelly monte cristo sandwiches,['spread 1 tablespoons peanut butter on each o...,"['creamy peanut butter', 'strawberry jelly', '..."
152789,parmesan baked asparagus,"['preheat ovento 425 f', 'place the asparagus ...","['asparagus', 'extra virgin olive oil', 'seaso..."
95519,grandmother s old time tea cakes,['mix shortening and sugar together with pastr...,"['flour', 'sugar', 'eggs', 'buttermilk', 'shor..."
121315,lemon chicken with olives,"['fry off chicken until browned , about 5 minu...","['boneless chicken thighs', 'olive oil', 'onio..."


#### Concatenate steps into single string.

In [3]:
df_concat = df_dropped.copy()

# Pandas is already treating it like a string for whatever reason, so just make it "look" not like a list
df_concat['steps'] = df_concat['steps'].apply(lambda x: ''.join([str(i) for i in x if i not in ['[', ']', "'"]]))

df_concat.head()

,name,steps,ingredients
99795,grilled salmon with rosemary south beach phase i,"cut the fish into 4 equal-size portions, combi...","['salmon', 'olive oil', 'fresh lemon juice', '..."
155700,peanut butter and jelly monte cristo sandwiches,spread 1 tablespoons peanut butter on each of ...,"['creamy peanut butter', 'strawberry jelly', '..."
152789,parmesan baked asparagus,"preheat ovento 425 f, place the asparagus on a...","['asparagus', 'extra virgin olive oil', 'seaso..."
95519,grandmother s old time tea cakes,mix shortening and sugar together with pastry ...,"['flour', 'sugar', 'eggs', 'buttermilk', 'shor..."
121315,lemon chicken with olives,"fry off chicken until browned , about 5 minute...","['boneless chicken thighs', 'olive oil', 'onio..."


#### Transform ingredients into columns.

In [ ]:
df_ings = df_concat.copy()

# Do the same character removal as in the previous step, now for ingredients col
df_ings['ingredients'] = df_ings['ingredients'].apply(lambda x: ''.join([str(i).strip() for i in x if i not in ['[', ']', "'"]]))

# Split on commas to make it an actual list now
df_ings['ingredients'] = df_ings['ingredients'].apply(lambda x: x.split(','))

# Create an ingredient column (if doesn't exist) and set the cell value to 1 for each ingredient of each recipe
for index, row in df_ings.iterrows():
    for ingredient in row['ingredients']:
        df_ings.at[index, ingredient] = 1
        
# Repalce NaNs with 0
df_ings = df_ings.fillna(0)

# Drop the original ingredients list
df_ings = df_ings.drop(['ingredients'], axis=1)

print(df_ings.shape)
df_ings.head()

In [ ]:
df_ings.to_csv('processed/recipes_processed.csv', index=False)

print('done')